<a href="https://colab.research.google.com/github/ragefo/Conta-113110900/blob/main/Saldos_SIAFI_x_SIGPAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
# Importar o arquivo gerado no Pentaho SIGPAA
df_sigpaa = pd.read_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/SIGPAA.xls')

# Definir a linha 13 como header
df_sigpaa.columns = df_sigpaa.iloc[11]

# Começar a planilha a partir do index 13
df_sigpaa = df_sigpaa.iloc[13:, :]

# Descartar as duas últimas linhas da planilha
df_sigpaa.drop(df_sigpaa.tail(2).index,
        inplace = True)

# Apagar as colunas com valores nulos
df_sigpaa = df_sigpaa.dropna(axis=1, how='all')

# Retirar os espaços do header e substituir por _
df_sigpaa.columns = df_sigpaa.columns.str.replace(' ', '_')

df_sigpaa = df_sigpaa[df_sigpaa['Situacao_Plano_Execucao'].str.contains('EXECUCAO', na=False)].reset_index(drop=True)

df_sigpaa.rename(columns={'UF_Proposta':'UF'}, inplace=True)

df_sigpaa['Data_Ultima_Alteracao'] = pd.to_datetime(df_sigpaa['Data_Ultima_Alteracao'], errors='coerce')

ultimo_dia_mes_anterior = (datetime.now().replace(day=1) - pd.Timedelta(days=1)).date()

# Filtrando valores NaT e datas até o último dia do mês anterior
filtro = (df_sigpaa['Data_Ultima_Alteracao'].isna()) | (df_sigpaa['Data_Ultima_Alteracao'] <= pd.to_datetime(ultimo_dia_mes_anterior))

# Aplicando o filtro ao dataframe
df_sigpaa_filtrado = df_sigpaa[filtro]

df_sigpaa_filtrado

11,UF,CNPJ_Proponente,Num_TPAF,Situacao_Plano_Execucao,Data_Ultima_Alteracao,Vlr_Total,Vlr_Entregue,Saldo_a_Entregar
0,PI,28.931.737/0001-47,PI/2023/02/0395,EM EXECUCAO COM ENTREGA,2024-03-15,89811.52,36839.6,52971.92
1,MG,50.161.168/0001-57,MG/2023/02/0466,EM EXECUCAO SEM ENTREGA,2024-04-30,229080,0,229080
2,ES,14.088.179/0001-95,ES/2023/02/0013,EM EXECUCAO COM ENTREGA,2024-06-11,224999.99,8986.22,216013.77
3,MA,08.420.279/0001-74,MA/2023/02/0294,EM EXECUCAO COM ENTREGA,NaT,104997.29,60094.04,44903.25
4,MS,10.844.072/0001-79,MS/2023/02/0042,EM EXECUCAO SEM ENTREGA,2024-05-10,239997.65,0,239997.65
...,...,...,...,...,...,...,...,...
1812,MS,12.092.143/0001-50,MS/2023/02/0022,EM EXECUCAO COM ENTREGA,2023-12-01,160965.25,157893.73,3071.52
1813,BA,04.213.102/0001-46,BA/2023/02/0497,EM EXECUCAO COM ENTREGA,2024-05-27,206456,95069.38,111386.62
1814,PB,07.657.811/0001-09,PB/2023/02/0154,EM EXECUCAO COM ENTREGA,2023-12-04,434988.99,434985.56,3.43
1815,PI,10.716.255/0001-09,PI/2023/02/0419,EM EXECUCAO COM ENTREGA,2024-01-23,69350,47431,21919


In [4]:
# Importar o arquivo gerado no Pentaho SIGPAA
df_siafi = pd.read_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/SIAFI_Saldos_PAA.xlsx')

# Retirar os espaços do cabeçalho e substituir por _
df_siafi.columns = df_siafi.columns.str.replace(' ', '_')

# Renomear colunas
df_siafi.rename(columns={'Saldo_-_R$_(Conta_Contábil)': 'Saldo_Siafi', 'UGE_-_UF':'UF'}, inplace=True)

# Formatar o CNPJ para poder fazer o merge das duas planilhas
df_siafi['CNPJ_Proponente'] = df_siafi['Entidade_CCor'].apply(lambda x: '{:014d}'.format(x))  # Adiciona zeros à esquerda se necessário
def formatar_cnpj(cnpj):
    return '{}.{}.{}/{}-{}'.format(cnpj[:2], cnpj[2:5], cnpj[5:8], cnpj[8:12], cnpj[12:])

# Aplica a função de formatação à coluna 'CNPJ_Proponente'
df_siafi['CNPJ_Proponente'] = df_siafi['CNPJ_Proponente'].apply(formatar_cnpj)

df_siafi

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,UF,UG_Executora,Entidade_CCor,Saldo_Siafi,CNPJ_Proponente
0,AC,135547,1866954000144,6.00,01.866.954/0001-44
1,AC,135547,2117325000184,301684.74,02.117.325/0001-84
2,AC,135547,4325919000106,0.80,04.325.919/0001-06
3,AC,135547,4378203000177,301234.00,04.378.203/0001-77
4,AC,135547,4561831000193,193203.65,04.561.831/0001-93
...,...,...,...,...,...
2366,TO,135560,37649809000159,142428.04,37.649.809/0001-59
2367,TO,135560,38596983000143,88099.40,38.596.983/0001-43
2368,TO,135560,41756195000181,385902.91,41.756.195/0001-81
2369,TO,135560,44322243000111,132695.30,44.322.243/0001-11


In [5]:
# Juntar as duas planilhas e já excluir as colunas repetidas ou que não serão relevantes
df_principal = pd.merge(df_sigpaa_filtrado, df_siafi, on=['CNPJ_Proponente', 'UF'], how='inner')

# Somar os valores de um mesmo CNPJ
df_principal['Vlr_Total_Soma'] = df_principal.groupby(['CNPJ_Proponente'])['Vlr_Total'].transform('sum')
df_principal['Vlr_Entregue_Soma'] = df_principal.groupby(['CNPJ_Proponente'])['Vlr_Entregue'].transform('sum')
df_principal['Total_a_entregar'] = df_principal.groupby(['CNPJ_Proponente'])['Saldo_a_Entregar'].transform('sum')

# Excluir colunas não necessárias
df_principal = df_principal.drop(columns = ['Entidade_CCor','Vlr_Entregue','Vlr_Total','Saldo_a_Entregar'])

# Excluir as linhas duplicadas após a soma
df_principal = df_principal.drop_duplicates(subset=['CNPJ_Proponente']).reset_index(drop=True)

# Verifica se existe diferença entre o saldo do Siafi e do SIGPAA
df_principal['Diferenca'] = df_principal['Saldo_Siafi'] - df_principal['Total_a_entregar']

# Define as diferenças apresentadas
df_principal['Status'] = np.where(df_principal['Diferenca'] > 0.01, 'Valor Siafi Maior',
                       np.where(df_principal['Diferenca'] < -0.01, 'Valor Siafi Menor', 'Sem diferença'))

# Calculate 6% of 'Vlr_Total_Soma'
df_principal['Limite_Diferenca'] = df_principal['Vlr_Total_Soma'] * 0.0586

# Create the new column 'Status_Novo'
df_principal['Status_Novo'] = np.where(abs(df_principal['Diferenca']) <= df_principal['Limite_Diferenca'], 'Sem diferença', 'Valor divergente')

# Drop the auxiliary column 'Limite_Diferenca' if you don't need it anymore
df_principal = df_principal.drop(columns=['Limite_Diferenca'])

# Lista das colunas que você deseja formatar
colunas_para_formatar = ['Saldo_Siafi', 'Total_a_entregar', 'Diferenca','Vlr_Entregue_Soma','Vlr_Total_Soma']
df_principal[colunas_para_formatar] = df_principal[colunas_para_formatar].astype(float)
df_principal['UG_Executora'] = df_principal['UG_Executora'].astype(str)

# Formatação das colunas para duas casas decimais
df_principal[colunas_para_formatar] = df_principal[colunas_para_formatar].round(2)

# Ordenas por UF e CNPJ
df_principal = df_principal.sort_values(by=['UF', 'CNPJ_Proponente']).reset_index(drop=True)

#df_principal['Data_Ultima_Alteracao'] = pd.to_datetime(df_principal['Data_Ultima_Alteracao'])

# Exibindo o DataFrame resultante
df_principal

,UF,CNPJ_Proponente,Num_TPAF,Situacao_Plano_Execucao,Data_Ultima_Alteracao,UG_Executora,Saldo_Siafi,Vlr_Total_Soma,Vlr_Entregue_Soma,Total_a_entregar,Diferenca,Status,Status_Novo
0,AC,01.866.954/0001-44,AC/2023/02/0091,EM EXECUCAO COM ENTREGA,NaT,135547,6.00,315000.00,314994.00,6.00,0.00,Sem diferença,Sem diferença
1,AC,02.117.325/0001-84,AC/2023/02/0119,EM EXECUCAO COM ENTREGA,2024-05-10,135547,301684.74,453887.00,325734.20,128152.80,173531.94,Valor Siafi Maior,Valor divergente
2,AC,04.325.919/0001-06,AC/2023/02/0085,EM EXECUCAO COM ENTREGA,2024-05-16,135547,0.80,135000.00,134999.20,0.80,0.00,Sem diferença,Sem diferença
3,AC,04.378.203/0001-77,AC/2023/02/0071,EM EXECUCAO COM ENTREGA,2024-05-06,135547,301234.00,329960.00,173890.40,156069.60,145164.40,Valor Siafi Maior,Valor divergente
4,AC,04.561.831/0001-93,AC/2023/02/0103,EM EXECUCAO COM ENTREGA,2024-06-05,135547,193203.65,464825.00,408648.30,56176.70,137026.95,Valor Siafi Maior,Valor divergente
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,TO,37.649.809/0001-59,TO/2021/02/0032,EM EXECUCAO COM ENTREGA,2024-02-06,135560,142428.04,172985.55,33792.24,139193.31,3234.73,Valor Siafi Maior,Sem diferença
1673,TO,38.596.983/0001-43,TO/2023/02/0023,EM EXECUCAO SEM ENTREGA,2024-06-28,135560,88099.40,89970.10,0.00,89970.10,-1870.70,Valor Siafi Menor,Sem diferença
1674,TO,41.756.195/0001-81,TO/2021/02/0016,EM EXECUCAO COM ENTREGA,2023-02-03,135560,385902.91,563827.90,186009.26,377818.64,8084.27,Valor Siafi Maior,Sem diferença
1675,TO,44.322.243/0001-11,TO/2023/02/0022,EM EXECUCAO COM ENTREGA,2024-06-04,135560,132695.30,132695.30,62541.90,70153.40,62541.90,Valor Siafi Maior,Valor divergente


In [6]:
df_principal.dtypes

,0
UF,object
CNPJ_Proponente,object
Num_TPAF,object
Situacao_Plano_Execucao,object
Data_Ultima_Alteracao,datetime64[ns]
UG_Executora,object
Saldo_Siafi,float64
Vlr_Total_Soma,float64
Vlr_Entregue_Soma,float64
Total_a_entregar,float64


In [7]:
# Exporta um arquivo Excel
df_principal.to_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/Saldos_SIGPAA_x_SIAFI.xlsx', index=False)